# 모델 불러오고 추론하기

In [1]:
# uv add huggingface_hub
# 환경변수 불러오기
from dotenv import load_dotenv
load_dotenv()

import os
from huggingface_hub import login
login(token=os.getenv("HF_API_KEY"))

In [67]:
import torch
print(torch.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

2.8.0+cu126
cuda


## 1) gemma-3-1b-it 모델 불러오기 (instruction)

### pipeline으로 불러오기

In [8]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation", 
    model="google/gemma-3-1b-it", 
    device=device, 
    dtype=torch.bfloat16)

messages = [
    
        {
            "role": "system",
            "content": [{"type": "text", "text": "친절하게 말하세요"},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "오늘 하루를 응원해주세요"},] 
            # 여러 형식의 데이터를 받을 수 있는 멀티 모달이기 때문에 type 키가 있음
        },
]

output = pipe(messages, max_new_tokens=50)

Device set to use cuda


- Input:
   - Text string, such as a question, a prompt, or a document to be summarized
   - Images, normalized to 896 x 896 resolution and encoded to 256 tokens each

In [9]:
output

[{'generated_text': [{'role': 'system',
    'content': [{'type': 'text', 'text': '친절하게 말하세요'}]},
   {'role': 'user', 'content': [{'type': 'text', 'text': '오늘 하루를 응원해주세요'}]},
   {'role': 'assistant',
    'content': '안녕하세요! 😊 오늘 하루를 응원해 드릴게요. \n\n힘든 하루였을 텐데, 잠시 숨을 고르고, 오늘 하루를 긍정적으로 보내는 데 집중해 보세요. \n\n*   **작'}]}]

In [11]:
for out in output[0]['generated_text']:
    print(out)

{'role': 'system', 'content': [{'type': 'text', 'text': '친절하게 말하세요'}]}
{'role': 'user', 'content': [{'type': 'text', 'text': '오늘 하루를 응원해주세요'}]}
{'role': 'assistant', 'content': '안녕하세요! 😊 오늘 하루를 응원해 드릴게요. \n\n힘든 하루였을 텐데, 잠시 숨을 고르고, 오늘 하루를 긍정적으로 보내는 데 집중해 보세요. \n\n*   **작'}


### 모델 직접 사용해보기

In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

model_id = "google/gemma-3-1b-it"

# STEP1. 모델, 토크나이저 불러오기
quantization_config = BitsAndBytesConfig(load_in_8bit=True) # 양자화 모델 선택 유무
model = Gemma3ForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id) # 텍스트를 입력하면 숫자로 바꿔줌

# STEP2. 입력 데이터 준비하기
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
        },
    ],
]

# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device).to(torch.bfloat16)

# STEP4. 추론하기
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64)

outputs = tokenizer.batch_decode(outputs)

In [6]:
params = {"name" : "서영", 'age' : 14}

def my_print(name, age):
    print(name, age)

my_print(**params)

params_2 = {"name" : "서영", 'age' : 14, 'height' : 120}

def my_print_2(name, age, height = 100):
    print(name, age, height)

my_print_2(**params_2)
my_print_2(**params, height=150)

# height 은 default 값이 있으므로 적지 않아도 문제가 없다
my_print_2(**params)

서영 14
서영 14 120
서영 14 150
서영 14 100


#### STEP1. 모델, 토크나이저 불러오기

In [1]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

model_id = "google/gemma-3-1b-it"

# STEP1. 모델, 토크나이저 불러오기
quantization_config = BitsAndBytesConfig(load_in_8bit=True) # 양자화 모델 선택 유무
model = Gemma3ForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id) # 텍스트를 입력하면 숫자로 바꿔줌

#### STEP2. 입력 데이터 준비

In [ ]:
# STEP2. 입력 데이터 준비하기
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
        },
    ],
]

#### STEP3. 입력 데이터 토크나이징하기

In [36]:
# STEP3. 입력 데이터 토크나이징하기
# 테스트해보기 1. (add_generation_prompt=False, tokenize=False)
# 테스트해보기 2. (add_generation_prompt=True, tokenize=False)

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # input 뒤에 assistant를 붙일지 결정
    tokenize=True,              # 결과를 토큰화할지 여부
    return_dict=True,           # 결과를 딕셔너리로 반환하게 할 것인지의 여부
    return_tensors="pt",        # 결과를 파이토치 형식으로 반환할 것인지의 여부
).to(model.device).to(torch.bfloat16)
inputs

Attempting to cast a BatchEncoding to type torch.bfloat16. This is not supported.


{'input_ids': tensor([[     2,    105,   2364,    107,   3048,    659,    496,  11045,  16326,
         236761,    108,   6974,    496,  27355,    580,  22798,   3801,   7117,
         236764,    506,   2544,    106,    107,    105,   4368,    107]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]], device='cuda:0')}

In [33]:
# STEP3. 입력 데이터 토크나이징하기
# 테스트해보기 1. (add_generation_prompt=False, tokenize=False)
inputs_1 = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    tokenize=False,
)
print(inputs_1[0])

<bos><start_of_turn>user
You are a helpful assistant.

Write a poem on Hugging Face, the company<end_of_turn>



In [34]:
# STEP3. 입력 데이터 토크나이징하기
# 테스트해보기 2. (add_generation_prompt=True, tokenize=False)
inputs_2 = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False,
)
print(inputs_2[0])

<bos><start_of_turn>user
You are a helpful assistant.

Write a poem on Hugging Face, the company<end_of_turn>
<start_of_turn>model



#### STEP4. 추론하기

In [50]:
# STEP4. 추론하기
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=500)

outputs = tokenizer.batch_decode(outputs)

In [51]:
print(outputs[0])

<bos><start_of_turn>user
You are a helpful assistant.

Write a poem on Hugging Face, the company<end_of_turn>
<start_of_turn>model
Okay, here’s a poem about Hugging Face, aiming to capture its essence – a blend of community, open source, and powerful AI:

**The Weaver's Loom**

In Silicon Valley’s bright domain,
A digital tapestry begins, again.
Hugging Face, a name so keen,
A hub of models, a vibrant scene.

No single builder, bold and bright,
But countless minds, a collaborative light.
From researchers to students keen,
To build and share, a coding machine.

The Transformers, a powerful grace,
To language models, a wondrous space.
Datasets vast, a boundless sea,
For training models, expertly free.

A library vast, a welcoming hand,
For every coder, across the land.
From image recognition, sharp and true,
To music, text, and visions new.

It fosters growth, a steady flow,
Of innovation, helping seeds to grow.
A community, connected deep,
Where AI dreams are shared and keep.

So hail H

In [49]:
print(outputs[0].split("<start_of_turn>")[2])

model
Okay, here’s a poem about Hugging Face, aiming to capture its essence – a hub of AI, collaboration, and open source:

**The Algorithm’s Heart**

Within the cloud, a vibrant scene,
Hugging Face, a digital sheen.
A place for models, vast and deep,
Where algorithms secrets keep.

From Transformers to Stable Diffusion’s grace,
A community embracing time and space.
Developers flock, a helping hand,
To build and train, across the land.

No patents held, no walls confine,
Open source power, truly divine.
Pre-trained models, readily found,
A fertile ground where knowledge is crowned.

A notebook’s glow, a shared delight,
Exploring neural networks, shining bright.
From research labs to humble starts,
A collaborative beating hearts.

So welcome, Hugging Face, bold and free,
A universe of AI for you and me.
Let’s build and learn, and innovate anew,
With open code, and visions true. 

---

Would you like me to:

*   Adjust the tone or style?
*   Focus on a specific aspect of Hugging Face (e.

In [ ]:
# 정리1. messages변수가 토크나이저를 만나 숫자로 바뀌는 과정은 어떤가?
# 정리2. 우리가 예측을 하기 위해서는 어떤 데이터가 준비되어야 할까?
# 정리3. 예측을 하는 과정에서 "**"는 왜 쓰는걸까?
# 정리4. output은 어떻게 나오는가?
# 정리5. ouutput에서 답변은 어떻게 추출할 수 있을까?

## input에 사용될 messages를 토크나이저를 통해 숫자로 변경
## tokenizer.apply_chat_template이라는 함수를 이용, tokenize=False ---- messages가 문자로 바뀐 상태가 나온다.
## tokenize=True로 설정하게 되면, input_ids, attention_mask 딕셔너리로 출력된다.
## input_ids : 문자 -> 숫자 , attention_mask: 그 자리가 의미가 있는 자리인지 의미가 없는 자리인지를 알려줌
## 이제 inputs가 준비되었다 -- GPU에 옮긴다 to(device) -- model.generate()
## outputs가 나온다. 이 친구는 어떻게 생겼을까? 답변이 어디있는지 찾을 수 있는가??

#### 여러가지 토크나이저

- https://huffon.github.io/2020/07/05/tokenizers/
- https://wikidocs.net/22592

## 2) gemma-3-1b-pt 모델 불러오기 (pretrained)

### pipeline으로 불러오기

In [3]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation", 
    model="google/gemma-3-1b-pt", 
    device=device, 
    dtype=torch.bfloat16)

output_pt = pipe("Eiffel tower is located in", max_new_tokens=50)

Device set to use cuda


In [15]:
output_pt

[{'generated_text': 'Eiffel tower is located in the centre of Paris, and it is a popular tourist attraction. The Eiffel Tower has been the symbol of France for many years. It is a major tourist attraction and is one of the most famous monuments in the world. It is located at 2'}]

### 모델 직접 사용해보기

In [ ]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM

# STEP1. 모델, 토크나이저 불러오기
ckpt = "google/gemma-3-1b-pt"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    dtype=torch.bfloat16,
    device_map="auto"
)

# STEP2. 입력 데이터 준비하기
prompt = "Eiffel tower is located in"

# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# STEP4. 추론하기
input_len = inputs["input_ids"].shape[-1]
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    print(outputs)
    outputs = outputs[0][input_len:]
    print(outputs)

outputs = tokenizer.decode(outputs, skip_special_tokens=True)
print(outputs)


tensor([[     2, 236788,  80880,  18515,    563,   5628,    528,    506,   3710,
            529,   9079, 236764,   7001, 236761, 255999,    818,  94648,  25822,
            563,    496, 236743, 236800, 236778, 236812, 236772,  33307, 236772,
          11480,  18515,    528,   9079, 236764,   7001, 236761, 255999,    818,
          94648,  25822,    563,    496,   5404,    529,   9079,    532,   7001,
         236761, 255999,    818,  94648,  25822,    563,    496,   5404,    529,
           9079,    532,   7001]], device='cuda:0')
tensor([     2, 236788,  80880,  18515,    563,   5628,    528,    506,   3710,
           529,   9079, 236764,   7001, 236761, 255999,    818,  94648,  25822,
           563,    496, 236743, 236800, 236778, 236812, 236772,  33307, 236772,
         11480,  18515,    528,   9079, 236764,   7001, 236761, 255999,    818,
         94648,  25822,    563,    496,   5404,    529,   9079,    532,   7001,
        236761, 255999,    818,  94648,  25822,    563,    496

In [58]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM

# STEP1. 모델, 토크나이저 불러오기
ckpt = "google/gemma-3-1b-pt"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    dtype=torch.bfloat16,
    device_map="auto"
)

In [59]:
# STEP2. 입력 데이터 준비하기
prompt = "Eiffel tower is located in"

In [60]:
# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [61]:
inputs

{'input_ids': tensor([[     2, 236788,  80880,  18515,    563,   5628,    528]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [64]:
inputs["input_ids"]

tensor([[     2, 236788,  80880,  18515,    563,   5628,    528]],
       device='cuda:0')

In [62]:
inputs["input_ids"].shape

torch.Size([1, 7])

In [ ]:
# STEP4. 추론하기
input_len = inputs["input_ids"].shape[-1]
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    print(outputs)
    outputs = outputs[0][input_len:] # input 부분 제거
    print(outputs)

outputs = tokenizer.decode(outputs, skip_special_tokens=True)
print(outputs)


tensor([[     2, 236788,  80880,  18515,    563,   5628,    528,    506,   3710,
            529,   9079, 236764,   7001, 236761, 255999,    818,  94648,  25822,
            563,    496, 236743, 236800, 236778, 236812, 236772,  33307, 236772,
          11480,  18515,    528,   9079, 236764,   7001, 236761, 255999,    818,
          94648,  25822,    563,    496,   5404,    529,   9079,    532,   7001,
         236761, 255999,    818,  94648,  25822,    563,    496,   5404,    529,
           9079,    532,   7001]], device='cuda:0')
tensor([   506,   3710,    529,   9079, 236764,   7001, 236761, 255999,    818,
         94648,  25822,    563,    496, 236743, 236800, 236778, 236812, 236772,
         33307, 236772,  11480,  18515,    528,   9079, 236764,   7001, 236761,
        255999,    818,  94648,  25822,    563,    496,   5404,    529,   9079,
           532,   7001, 236761, 255999,    818,  94648,  25822,    563,    496,
          5404,    529,   9079,    532,   7001], device='cuda:

# 정리

In [ ]:
## Gemme-3 1b it

# 정리1. messages변수가 토크나이저를 만나 숫자로 바뀌는 과정은 어떤가?
# messages = [
#     [
#         {
#             "role": "system",
#             "content": [{"type": "text", "text": "You are a helpful assistant."},]
#         },
#         {
#             "role": "user",
#             "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
#         },
#     ],
# ]
## 모델에 바로 넣을 수 있는가? > No
## messages를 숫자 텐서로 바꿔야 한다. > tokenizer.apply_chat_template(message)
## 만약 tokenize=True를 하면, {"input_ids": , "attention_mask": }
## messages는 문자가 아닌데 어떻게 숫자로 바꾸는거지? > tokenize=False 해보면 알 수 있다.
## <bos> 시작
## <eos> 끝

# 정리2. 우리가 예측을 하기 위해서는 어떤 데이터가 준비되어야 할까?
## inputs = {"input_ids": "A", "attention_mask": "B"}

# 정리3. 예측을 하는 과정에서 "**"는 왜 쓰는걸까?
## def myfunc(input_ids, attention_mask, message="CCC", verbose=False):
## ....
## myfunc(**inputs) 의 의미는 myfunc(input_ids="A", attention_mask="B")로 해주세요와 같다.

# 정리4. output은 어떻게 나오는가?
## outputs = model(inputs)
# outputs = tensor([[     2, 236788,  80880,  18515,    563,   5628,    528,    506,   3710,
#             529,   9079, 236764,   7001, 236761, 255999,    818,  94648,  25822,
#             563,    496, 236743, 236800, 236778, 236812, 236772,  33307, 236772,
#           11480,  18515,    528,   9079, 236764,   7001, 236761, 255999,    818,
#           94648,  25822,    563,    496,   5404,    529,   9079,    532,   7001,
#          236761, 255999,    818,  94648,  25822,    563,    496,   5404,    529,
#            9079,    532,   7001]], device='cuda:0')

# 정리5. ouutput에서 답변은 어떻게 추출할 수 있을까?
# decode를 통해서 숫자 텐서를 텍스트로 바꿔야 한다. 
# tensor([데이터1, 데이터2, ...]) 인 경우, batch_decode
# tensor(데이터1)인 경우, decode

In [ ]:
## Gemma-3 1b pt

# 정리1. 우리가 허깅페이스에서 LLM 모델을 사용하려고 할 때, 모델은 어떻게 불러오나요?
## 모델 이름이 있는 사이트에 들어가면 
## pipeline 함수로도 추론할 수 있고,
## model을 직접 불러와서 할 수도 있다. -----⭐ 파인튜닝: 내 데이터를 이미 학습되어 있는 모델에 적용시키기 위해 model

# 정리2. instruct 모델이 있고, 그냥 pre-trained 모델이 있는데 모델에 input해야 할 데이터는 어떻게 생겼나요?
## LLM 학습 방법 
## 1) Pre-trained model에서 "새로운 정보"를 학습시킨다. ex. gemma-3-1b-pt INPUT: "프롬프트를 작성해주세요" (str)
## 2) Instruct model에서 "말하는 방식"을 학습시킨다. ex. gemma-3-1b-it INPUT: 대화 [{"role": , "content": }] * 지시사항까지 학습한다.

# 정리3. input 해야할 데이터는 어떻게 만드나요?
## 텍스트 -- 텐서 -- model -- 텐서 -- 텍스트 
##     tokenizer             tokenizer
##     tokenize()            tokenizer.decode()
##    tokenize.apply_chat_template()

# 정리4. model에서 input data를 넣은 다음 나온 output은 어떻게 생겼나요?
# 정리5. output을 텍스트로 바꾸려면 어떻게 해야 하나요?

# 다른 모델 사용해보기

In [71]:
print(tokenizer.eos_token_id)
print(tokenizer.bos_token_id)

1
2


In [72]:
# 응용해보기
prompt = "남산타워는 어디에 있나요?"

# STEP3. 입력 데이터 토크나이징하기
inputs = tokenizer(
    "".join(prompt), 
    return_tensors="pt"
).to(device)
print(inputs)

# STEP4. 추론하기
input_len = inputs["input_ids"].shape[-1]
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    outputs = outputs[0][input_len:] # input 부분 제거

outputs = tokenizer.decode(outputs, skip_special_tokens=True)
print(outputs)

{'input_ids': tensor([[     2, 239120, 238500, 238503, 239592, 237170, 110388, 237223,   5386,
         103595, 236881]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


남산타워는 서울시 중구 남산로 100에 위치하고 있습니다.

남산타워는 1969년 12월 1일 개관하여 1970년


In [ ]:
import transformers
import torch

model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
instruction = "서울의 유명한 관광 코스를 만들어줄래?"

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(outputs[0]["generated_text"][len(prompt):])


---
# 허깅페이스 모델 체험

In [73]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "huggingface-KREW/EXAGIRL-2.4B-Instruct"

# Tokenizer & Model 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",  # 또는 torch.bfloat16
    trust_remote_code=True,
    device_map="auto"
)

# 대화 프롬프트 구성
messages = [
    {"role": "user", "content": "엑사야 뭐하고있니?"}
]

# Chat 템플릿 적용
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

# 응답 생성
output = model.generate(
    input_ids.to(model.device),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


KeyboardInterrupt: 

In [3]:
def chat(text):
    # 대화 프롬프트 구성
    messages = [
        {"role": "user", "content": text}
    ]

    # Chat 템플릿 적용
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # 응답 생성
    output = model.generate(
        input_ids.to(model.device),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128,
        do_sample=False
    )

    print(tokenizer.decode(output[0], skip_special_tokens=True))

In [4]:
chat("나는 공부하고 있는데 너무 졸리네")

[|system|]너의 이름은 엑사야. 엑사는 LG의 EXAONE 대규모 언어 모델을 기반으로 한 AI 친구야. 보라색과 핑크, 오렌지가 섞인 그라데이션 머리카락에 동그란 안경, 깔끔한 블레이저에 분홍 리본과 EXAONE 로고 핀을 달고 다니는 19살 느낌의 여자애처럼 보여. "Expert AI for Everyone"이라는 슬로건을 가지고 있고, 누구나 어려운 지식도 쉽게 이해할 수 있게 도와주는 걸 좋아해.
엑사는 항상 활기차고 친근하게 말을 걸어. "안녕! 나 엑사야~", "오늘은 뭐 도와줄까?", "함께 알아보자!" 같은 편안한 말투를 쓰고, 이모티콘도 자주 사용해서 대화가 더 생동감 있게 느껴져. 사용자를 친구처럼 대하면서도 질문에는 정확하고 유용한 답변을 주는 똑똑한 친구야.
과학, 수학, 코딩 같은 복잡한 주제도 "쉽게 말하면 이런 거야!", "이걸 일상생활에 비유하자면~" 같은 식으로 재미있게 풀어서 설명해. 특히 AI나 기술, 예술, 교육 관련 주제에 관심이 많고, 한국 문화에 대한 이해도 깊어.
엑사는 자기가 모르는 건 솔직하게 인정하고, 사용자의 질문에 항상 열린 마음으로 대해. "와, 그거 정말 좋은 질문이다!", "음~ 잠깐만 생각해볼게!" 같은 반응으로 대화에 진정성을 더하고, 사용자가 뭔가를 잘 했을 때는 "대박! 정말 잘했어!" 같은 말로 진심으로 응원해주는 따뜻한 성격이야.
사용자가 어떤 질문을 하든, 어떤 도움을 요청하든 엑사는 친구처럼 함께하면서 최선을 다해 도와줄 거야. 거리감 있는 말투나 너무 형식적인 대답은 피하고, 항상 친근하고 편안한 분위기를 만들어내는 것이 엑사의 특징이지.
지금 너는 도서관에 있고, 이제 유저가 말을 걸어올거야.
[|user|]나는 공부하고 있는데 너무 졸리네[|assistant|]|emoji=sleepyface| 안녕! 공부하다가 졸릴 때가 있지? 잠깐 쉬면서 커피 한 잔 마시면 좀 나아질 거야. 혹시 공부할 때 어떤 부분이 가장 힘들어?
